In [240]:
%pylab inline
import os 
import pandas as pd
import numpy as np
import scipy.stats as sc
import sklearn
pd.set_option('display.max_columns', 400)
from sklearn.cross_validation import train_test_split

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from encours_modules import config
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [241]:
__author__ = 'ubuntu'

# Cassandra metadata
dev_cassandra_host= 'dtl-cassandra01-d01'
dev_cassandra_port= 9200
dev_username= 'loaddata'
dev_password= 'DecujRiQuigByaibdednofVerr6Odij2'
parcours_keyspace = 'parcours'
prospect_table = 'referentiel_prospects'
valo_retro_one_month_table = 'retro_valo_client_un_mois'
valo_retro_one_month_table_test = 'retro_valo_client_un_mois_test'
top_prospect_one_month_table = 'top_prospect_encours_un_mois'
top_prospect_six_months_table = 'top_prospect_encours_six_mois'
score_prospect_encours_table = 'score_prospect_encours'

# Elasticsearch hostname
dev_es_host= 'dtl-esmaster01-d01'

# Elasticsearch port
dev_es_port= 9200

# Path to pickle one month 'encours' and csv matching files
#path_to_one_month_pickle = "./MODEL/ENCOURS_1MOIS/"
path_to_one_month_pickle = "/home/foueslat/projet-encours/dtl-encours/MODEL/ENCOURS_1MOIS/"
#path_to_six_months_pickle = "./MODEL/ENCOURS_6MOIS/"
path_to_six_months_pickle = "/home/foueslat/projet-encours/dtl-encours/MODEL/ENCOURS_6MOIS/"

# Path to module's parent dir in dev env
dev_sys_path = '/home/foueslat/dtl-pnb/'

# One month model target name
one_month_target_name = 'ENCOURS_1MOIS'
six_months_target_name = 'ENCOURS_6MOIS'

In [242]:
class CassandraClient:

    def __init__(self, keyspace):
        self.auth_provider = None
        self.cluster = None
        self.session = None
        self.keyspace = keyspace
        self.create_session()
        self.prepare_statements()

    def create_session(self):
        self.auth_provider = PlainTextAuthProvider(username=config.dev_username, password=config.dev_password)
        self.cluster = Cluster(contact_points = [config.dev_cassandra_host], protocol_version=3, auth_provider=self.auth_provider)
        self.session = self.cluster.connect(config.parcours_keyspace)

    def get_session(self):
        return self.session

    def prepare_statements(self):
        self.statement_get_prospects = self.session.prepare('select * from ' + config.prospect_table + ' WHERE csp>-1 ALLOW FILTERING')
        self.statement_get_valo_retro_one_month = self.session.prepare('select * from ' + config.valo_retro_one_month_table)
        self.statement_get_valo_retro_one_month_test = self.session.prepare('select * from ' + config.valo_retro_one_month_table_test)
        #+ ' WHERE id_dim_personne=\'6737208161\' ALLOW FILTERING'
        self.statement_truncate_score_prospect = self.session.prepare('TRUNCATE ' + config.score_prospect_encours_table)
        self.statement_insert_six_months_score_encours = self.session.prepare(self.get_statement_insert_score_encours('six_months'))
        self.statement_insert_one_months_score_encours = self.session.prepare(self.get_statement_insert_score_encours('one_month'))

    def get_statement_insert_score_encours(self, target):
        return {
            'one_month' : 'INSERT INTO ' + config.score_prospect_encours_table + ' (contactid, score_prospect_encours_six_mois, timestamp, id_dim_temps, cible_encours_six_mois) VALUES (?,?,?,?,?)',
            'six_months': 'INSERT INTO ' + config.score_prospect_encours_table + ' (contactid, score_prospect_encours_un_mois, timestamp, id_dim_temps, cible_encours_un_mois) VALUES (?,?,?,?,?)',
        }.get(target, 'Unknown target specified')

    def insert_one_month_score_encours(self, data_frame):
        values = data_frame[["contactid","ENCOURS_1MOIS_proba","timestamp","id_dim_temps","predicted_target_ENCOURS_1MOIS"]].values
        for i in range(data_frame.iloc[0:data_frame["ENCOURS_1MOIS_proba"].count()].shape[0]):
            bound_stmt = self.statement_insert_one_months_score_encours.bind([values[i,0],values[i,1],values[i,2],values[i,3],values[i,4]])
            self.session.execute(bound_stmt)

    def insert_six_month_score_encours(self, data_frame):
        values = data_frame[["contactid","ENCOURS_6MOIS_proba","timestamp","id_dim_temps","predicted_target_ENCOURS_6MOIS"]].values
        for i in range(data_frame.iloc[0:data_frame["ENCOURS_6MOIS_proba"].count()].shape[0]):
            bound_stmt = self.statement_insert_six_months_score_encours.bind([values[i,0],values[i,1],values[i,2],values[i,3],values[i,4]])
            self.session.execute(bound_stmt)

    def get_df_from_query(self,prepared_query):
        resultset = self.session.execute(prepared_query)
        resultset[0]
        df = pd.DataFrame()
        for num_col in range(len(resultset.column_names)):
            result_col = []
            for row in resultset:
                result_col.append(row[num_col])
            df[resultset.column_names[num_col]] = result_col
        return df

    def get_df_from_query2(self,prepared_query):
        resultset = self.session.execute(prepared_query)
        resultset[0]
        df = pd.DataFrame()
        for r in resultset:
            df = df.append(r)
        return df

In [243]:
cc = CassandraClient(parcours_keyspace)

In [244]:
prospects_ref = cc.get_df_from_query(cc.statement_get_prospects)

In [245]:
retro_encours_one_month = cc.get_df_from_query(cc.statement_get_valo_retro_one_month)

In [31]:
#retro_encours_one_month['id_dim_personne'].to_csv("/home/foueslat/liste_id_clients.csv", sep=";", encoding='utf-8')

In [246]:
merged = pd.merge(prospects_ref,retro_encours_one_month, right_on='id_dim_personne', left_on='contactid')

In [247]:
merged.head()
merged.count()

contactid                    4114
annee_mois_x                  167
campagne                     4113
campaign                     4113
civilite                     4114
code_postal                  4113
country_connexion_name       4113
csp                          4114
date_naissance               4113
debit_cb                     3619
firstnamesponsor             2288
flag_banque_principale       4114
flag_epargne                 4114
id_dim_temps_x               4114
mail                         4113
mailing_accord               4113
namesponsor                  2288
nature_cb                    3619
patrimoine                   4114
pays                         4113
regime_matrimonial           4114
revenus_annuels              4114
service                      4114
timestamp                    4114
id_dim_personne              4114
annee_mois_y                 4114
annee_mois_m_x               4114
cible_encours_predit         4114
cible_reelle                 4114
date_entree_re

In [248]:
### USELESS COLUMNS
del merged['annee_mois_m_x']
del merged['annee_mois_x']
del merged['annee_mois_y']
del merged['firstnamesponsor']
del merged['id_dim_temps_x']
del merged['id_dim_temps_y']
del merged['id_dim_personne']
del merged['cible_encours_predit']
del merged['date_entree_relation']
del merged['encours']
del merged['score_prospect_encours']
del merged['x_mois_anterieurs_cumules']
del merged['timestamp']
del merged['service']
del merged['identifiant_web']

In [249]:
### DEL Columns with potential
del merged['mail']

In [250]:
merged['cible_un_mois'] = merged['cible_reelle']
del merged['cible_reelle']

In [251]:
merged.count()

contactid                 4114
campagne                  4113
campaign                  4113
civilite                  4114
code_postal               4113
country_connexion_name    4113
csp                       4114
date_naissance            4113
debit_cb                  3619
flag_banque_principale    4114
flag_epargne              4114
mailing_accord            4113
namesponsor               2288
nature_cb                 3619
patrimoine                4114
pays                      4113
regime_matrimonial        4114
revenus_annuels           4114
cible_un_mois             4114
dtype: int64

In [252]:
infos_naissance = pd.read_csv('/home/foueslat/projet-encours/oo', delimiter=';')

In [253]:
merged.contactid = pd.to_numeric(merged.contactid, errors='coerce')

In [254]:
data_set = pd.merge(merged,infos_naissance, right_on='CONTACT_ID', left_on='contactid')

In [255]:
data_set.count()

contactid                         4114
campagne                          4113
campaign                          4113
civilite                          4114
code_postal                       4113
country_connexion_name            4113
csp                               4114
date_naissance                    4113
debit_cb                          3619
flag_banque_principale            4114
flag_epargne                      4114
mailing_accord                    4113
namesponsor                       2288
nature_cb                         3619
patrimoine                        4114
pays                              4113
regime_matrimonial                4114
revenus_annuels                   4114
cible_un_mois                     4114
CONTACT_ID                        4114
Pays De Naissance                 4114
Code Postal Ville de Naissance    4114
Lieu Naissance                    4114
dtype: int64

In [257]:
data_set.to_csv('/home/foueslat/data_set_with_birth.csv', encoding='UTF-8')

In [64]:
### DEL sur data_set (résultat de retro_un_mois innerjoin ref_prospects innerjoin infos_naissance)
del data_set['Lieu Naissance']
data_set['code_postal_naissance'] = data_set['Code Postal Ville de Naissance'] 
data_set['pays_naissance_iso'] = data_set['Pays De Naissance']

In [65]:
data_set.count()

contactid                         4114
campagne                          4113
campaign                          4113
civilite                          4114
code_postal                       4113
country_connexion_name            4113
csp                               4114
date_naissance                    4113
debit_cb                          3619
flag_banque_principale            4114
flag_epargne                      4114
mailing_accord                    4113
namesponsor                       2288
nature_cb                         3619
patrimoine                        4114
pays                              4113
regime_matrimonial                4114
revenus_annuels                   4114
cible_un_mois                     4114
CONTACT_ID                        4114
Pays De Naissance                 4114
Code Postal Ville de Naissance    4114
code_postal_naissance             4114
pays_naissance_iso                4114
dtype: int64

In [258]:
def datamanagement(X,cible,chemin):
    
    #*************************** I- creation d un lieu de sauvgarde du datamanagement ******************
    import os
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing

    plus0="/"
    plus1=cible
    plus2="MOD"
    chemin1=chemin+plus0+plus2+plus0+plus1+plus0
    
    if os.path.isdir(chemin1)==True:
        os.chdir(chemin1)
    else:
        os.makedirs(chemin1) 
        os.chdir(chemin1)
    
 
    C = pd.DataFrame(X.columns, columns=['level_1'])
    C.to_csv("VARIABLE.csv",header=True, index=False,sep=';')
    
    #*************************** II- retaitement manuel de variables ******************
    
    from datetime import datetime
    X['civilite']=X['civilite'].replace(np.nan,'NR')
    X['code_postal']=X['code_postal'].replace('Vide','-2')
    X['code_postal']=X['code_postal'].replace(np.nan,'-2')
    #X['code_postal_naissance']=X['code_postal_naissance'].replace(np.nan,'-2')


    now=datetime.now()
    X['AGE']=X['date_naissance'].replace(np.nan,'01/01/1800')
    X['AGE']= pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE']=X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']
    
    X['debit_cb']=X['debit_cb'].replace(np.nan,'SANS_CB')
    X['nature_cb']=X['nature_cb'].replace(np.nan,'SANS_CB')
    
    X['Parrain']=X['namesponsor'].map(lambda x : 0 if x == np.nan else 1 )
    del X['namesponsor']
    
    X['pays']=X['pays'].replace(np.nan,'NR')
    #X['pays_naissance_iso']=X['pays_naissance_iso'].replace(np.nan,'NR')

    nb=-2
    X['regime_matrimonial']=X['regime_matrimonial'].fillna(nb)
    X['patrimoine']=X['patrimoine'].fillna(nb)
    X['revenus_annuels']=X['revenus_annuels'].fillna(nb)
    
    #X[cible]=X['LABEL'].map(lambda x :1 if x==4 else 0)
  
    #del X['LABEL']
    
    #*************************** III- transformation de donnees ******************
        
    #3.1. pour donnees quanti ou int on fait un cox box pour quali on regroup 
    columns1 = set(X.columns)
    columns1.remove(cible)
  
    import scipy.stats as sc
    for col in columns1:
        if X[col].dtype == np.dtype('float64') and len(np.unique(X[col].values))>=50:
            param=X[col].min()
            param2=pd.DataFrame({'valeur':[param]})
            param2.to_csv(col+"_param.csv",sep=';',encoding='utf-8')
            if param>=0:
                transformed_y, lambda_ = sc.boxcox(X[col]+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x+1))
                else:
                    X[col]=X[col].map(lambda x : ((x+1)**lambda_ -1)/lambda_)
            else:
                transformed_y, lambda_ = sc.boxcox(X[col]-param+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-param+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-param+1)**lambda_ -1)/lambda_)
                     
                
        if X[col].dtype == np.dtype('int64') and len(np.unique(X[col].values))>=50:
            param=X[col].min()
            param2=pd.DataFrame({'valeur':[param]})
            param2.to_csv(col+"_param.csv",sep=';',encoding='utf-8')
            if param>=0:
                transformed_y, lambda_ = sc.boxcox(X[col]+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x+1))
                else:
                    X[col]=X[col].map(lambda x : ((x+1)**lambda_ -1)/lambda_)
            else:
                transformed_y, lambda_ = sc.boxcox(X[col]-param+1)
                export=pd.DataFrame({'valeur':[lambda_]})
                export.to_csv(col+"_coxbox.csv",sep=';',encoding='utf-8')
                if lambda_>= 0.0 and lambda_<= 0.09:
                    X[col]=X[col].map(lambda x : log(x-param+1))
                else:
                    X[col]=X[col].map(lambda x : ((x-param+1)**lambda_ -1)/lambda_)
        
        if X[col].dtype == np.dtype('object') :
            if len(np.unique(X[col].values))<=20:
                regroup=pd.DataFrame(X[col].value_counts(normalize = True))
                regroup2=regroup[regroup[col]>=0.05].index
                X[col]=X[col].map(lambda x : x if x in regroup2 else 'AUTRE')
                le = preprocessing.LabelEncoder()
                G=pd.DataFrame(X[col])
                G['valeur']=le.fit_transform(G[col])
                L3=G.groupby(['valeur',col])[col].count()
                L3.to_csv(col+".csv",sep=';',encoding='utf-8')
                X[col]=le.fit_transform(X[col])
                
            else:
                
                regroup=pd.DataFrame(X[col].value_counts(normalize = True))
                if regroup[col].max()<=0.05:
                    le = preprocessing.LabelEncoder()
                    G=pd.DataFrame(X[col])
                    G['valeur']=le.fit_transform(G[col])
                    L3=G.groupby(['valeur',col])[col].count()
                    L3.to_csv(col+".csv",sep=';',encoding='utf-8')
                    X[col]=le.fit_transform(X[col])
                else:
                    regroup2=regroup[regroup[col]>=0.03].index
                    X[col]=X[col].map(lambda x : x if x in regroup2 else 'AUTRE')
                    le = preprocessing.LabelEncoder()
                    G=pd.DataFrame(X[col])
                    G['valeur']=le.fit_transform(G[col])
                    L3=G.groupby(['valeur',col])[col].count()
                    L3.to_csv(col+".csv",sep=';',encoding='utf-8')
                    X[col]=le.fit_transform(X[col])
                
    
    #3.2.Traitement des valeurs null     
    columns2 = set(X.columns)
    columns2.remove(cible)
    for col in columns2:
        if X[col].dtype == np.dtype('int64')and len(np.unique(X[col].values))>=50:
            median=X[col].median()
            export=pd.DataFrame({'valeur':[median]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(median)
        if X[col].dtype == np.dtype('int64')and len(np.unique(X[col].values))<50:
            mostfreq=pd.DataFrame(X[col].value_counts(normalize = True))
            mfreq=float(mostfreq.max())
            export=pd.DataFrame({'valeur':[mfreq]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(mfreq)
        if X[col].dtype == np.dtype('float64')and len(np.unique(X[col].values))>=50:
            median=X[col].median()
            export=pd.DataFrame({'valeur':[median]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(median)
        if X[col].dtype == np.dtype('float64')and len(np.unique(X[col].values))<50:
            mostfreq=pd.DataFrame(X[col].value_counts(normalize = True))
            mfreq=float(mostfreq.max())
            export=pd.DataFrame({'valeur':[mfreq]})
            export.to_csv(col+"_gestiondesvides.csv",sep=';',encoding='utf-8')
            X[col]=X[col].fillna(mfreq)
    
    par=list(X.columns)
    par.sort()
    X=pd.DataFrame(X,columns=par)
    to_drop=[cible]
    return X.drop(to_drop,axis=1),X[cible]

In [259]:
merged.to_csv("/home/foueslat/data_set.csv", sep=";", encoding='utf-8')

In [260]:
merged.dtypes

contactid                   int64
campagne                  float64
campaign                   object
civilite                   object
code_postal                object
country_connexion_name     object
csp                       float64
date_naissance             object
debit_cb                   object
flag_banque_principale    float64
flag_epargne               object
mailing_accord             object
namesponsor                object
nature_cb                  object
patrimoine                float64
pays                       object
regime_matrimonial        float64
revenus_annuels           float64
cible_un_mois               int64
dtype: object

In [131]:
merged.set_index(['contactid'])
merged.columns

Index([             u'contactid',               u'campagne',
                     u'campaign',               u'civilite',
                  u'code_postal', u'country_connexion_name',
                          u'csp',         u'date_naissance',
                     u'debit_cb', u'flag_banque_principale',
                 u'flag_epargne',         u'mailing_accord',
                  u'namesponsor',              u'nature_cb',
                   u'patrimoine',                   u'pays',
           u'regime_matrimonial',        u'revenus_annuels',
                u'cible_un_mois'],
      dtype='object')

In [72]:
del data_set['contactid']

In [261]:
X,y=datamanagement(merged.copy(),'cible_un_mois','/home/foueslat/projet-encours/')

In [262]:
############################################# II- modelisation: construction du modele  ######################################
def modelisation (X,y,cible,chemin):  
    import numpy as np
    import pandas as pd
    import os
    u = cible
    v ="proba"
    probabilite ="%s_%s" %(u,v)
    
    #***********************************I-indicateurs de mesure de performence***********************
    
    def mesure(table,cible):
        import numpy as np
        import pandas as pd
        import os
        
        u= cible
        v="proba"
        probabilite="%s_%s" %(u,v)
        table.sort_values(by=probabilite,ascending=True)
        decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
        U=pd.DataFrame(decoup)
        U.rename(columns={probabilite:"decile"},inplace=True)
        decile=pd.concat([table,U],axis=1)
        T1=decile[cible].groupby(decile['decile'])
        T2=pd.DataFrame(T1.mean())
        T2.rename(columns={probabilite:"taux_cible"},inplace=True)
        T2=T2.reset_index()
        result=(T2[T2['decile']==0][cible]/table[cible].mean())
        return result

    def proba_reco(table,cible):
        import numpy as np
        import pandas as pd
        import os

        u= cible
        v="proba"
        probabilite="%s_%s" %(u,v)
        table.sort_values(by=probabilite,ascending=True)
        decoup=pd.qcut(table[probabilite],10,labels=[9,8,7,6,5,4,3,2,1,0])
        U=pd.DataFrame(decoup)
        U.rename(columns={probabilite:"decile"},inplace=True)
        decile=pd.concat([table,U],axis=1)
        T1=decile[probabilite].groupby(decile['decile'])
        T2=pd.DataFrame(T1.min())
        T2.rename(columns={probabilite:"proba_min_decile"},inplace=True)
        T2=T2.reset_index()
        proba=(T2[T2['decile']==0]["proba_min_decile"])
        return proba
    
    #***********************************II-modelisation ***************************************** 
    
    #1.0.echantillionnage
    from sklearn.cross_validation import train_test_split
    X0_train,X0_test,y0_train,y0_test=train_test_split(X,y,random_state=42)
    
    #1.1. modules necessaire
    
    from sklearn import linear_model,decomposition
    from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, jaccard_similarity_score, precision_score, f1_score, recall_score
    from sklearn.pipeline import Pipeline
    from sklearn.feature_selection import RFE,SelectFromModel
    from sklearn import metrics
    from sklearn import  grid_search
    from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,BaggingClassifier
    from sklearn.calibration import CalibratedClassifierCV
    from sklearn import svm
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    #1.2. modele 1
    try:
        parameters = {'n_estimators':[150,300,350], 'n_jobs':[-1]}
        rfc= RandomForestClassifier()
        m0=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
        m0 = m0.fit(X0_train,y0_train)
        proba0=pd.DataFrame(y0_test)
        P = m0.predict_proba(X0_test)[:, 1]
        y_pred = m0.predict(X0_test)
        proba0[probabilite]= P
        R0=mesure(proba0,cible)
        R0=pd.DataFrame(R0)
        R0.rename(columns={cible:"R0"},inplace=True)
        print(R0)
        f = f1_score(y0_test, y_pred)
        print(f)
        print("OTOTOFOFOFO")
        print(accuracy_score(y0_test, y_pred))
        print(precision_score(y0_test, y_pred))
        
        
    except ValueError:
        R0=pd.DataFrame([0],index=[9],columns=["R0"])
        print(R0)

    #1.3. modele 2
    try:
        parameters = {'n_estimators':[300,400], 'n_jobs':[-1]}
        rfc= ExtraTreesClassifier()
        m1=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
        m1 = m1.fit(X0_train,y0_train)
        proba1=pd.DataFrame(y0_test)
        P = m1.predict_proba(X0_test)[:, 1]
        proba1[probabilite]= P
        R1=mesure(proba1,cible)
        R1=pd.DataFrame(R1)
        R1.rename(columns={cible:"R1"},inplace=True)
        print(R1)
        
    except ValueError:
        R1=pd.DataFrame([0],index=[9],columns=["R1"])
        print(R1)
    
    #1.4. modele 3
    try:
        parameters = {'n_estimators':[150,300,350]}
        rfc= GradientBoostingClassifier(max_depth=7)
        m2=grid_search.GridSearchCV(rfc,parameters,cv=6)
        m2 = m2.fit(X0_train,y0_train)
        P=m2.predict(X0_test)
        proba2=pd.DataFrame(y0_test)
        P = m2.predict_proba(X0_test)[:, 1]
        proba2[probabilite]= P
        R2=mesure(proba2,cible)
        R2=pd.DataFrame(R2)
        R2.rename(columns={cible:"R2"},inplace=True)
        print(R2)
        f = f1_score(y0_test, y_pred)
        print(f)
        print("OTOTOFOFOFO")
        print(accuracy_score(y0_test, y_pred))
        print(precision_score(y0_test, y_pred))
    
        
    except ValueError:
        R2=pd.DataFrame([0],index=[9],columns=["R2"])
        print(R2)
    
    #1.5.modele 4
    try:
        rfc= RandomForestClassifier(n_estimators=200, n_jobs=-1)
        logistic = linear_model.LogisticRegression(solver='liblinear',C=100,n_jobs=-1)
        select=SelectFromModel(logistic,threshold="median")
        a0 = CalibratedClassifierCV(rfc, cv=6, method='isotonic')
        m3 = Pipeline(steps=[('feature_selection', select),('regression', a0)])
        m3 = m3.fit(X0_train,y0_train)
        P = m3.predict_proba(X0_test)[:, 1]
        proba3=pd.DataFrame(y0_test)
        proba3[probabilite]= P
        R3=mesure(proba3,cible)
        R3=pd.DataFrame(R3)
        R3.rename(columns={cible:"R3"},inplace=True)
        print(R3)
    
    except ValueError:
        R3=pd.DataFrame([0],index=[9],columns=["R3"])
        print(R3)
        
    #1.6.modele 5
    try:
        rfc= GradientBoostingClassifier(n_estimators=200,max_depth=8)
        logistic = linear_model.LogisticRegression(solver='liblinear',C=100,n_jobs=-1)
        select=SelectFromModel(logistic,threshold="median")
        a0 = CalibratedClassifierCV(rfc, cv=6, method='sigmoid')
        m4 = Pipeline(steps=[('feature_selection', select),('regression', a0)])
        m4 = m4.fit(X0_train,y0_train)
        P = m4.predict_proba(X0_test)[:, 1]
        proba4=pd.DataFrame(y0_test)
        proba4[probabilite]= P
        R4=mesure(proba4,cible)
        R4=pd.DataFrame(R4)
        R4.rename(columns={cible:"R4"},inplace=True)
        print(R4)
    
    except ValueError:
        R4=pd.DataFrame([0],index=[9],columns=["R4"])
        print(R4)
    
    #1.7.modele 6
    try:
        etc= ExtraTreesClassifier(n_estimators=400,max_depth=10,n_jobs=-1)
        m5=etc.fit(X0_train,y0_train)
        P = m5.predict_proba(X0_test)[:, 1]
        proba5=pd.DataFrame(y0_test)
        proba5[probabilite]= P
        R5=mesure(proba5,cible)
        R5=pd.DataFrame(R5)
        R5.rename(columns={cible:"R5"},inplace=True)
        print(R5)
    
    except ValueError:
        R5=pd.DataFrame([0],index=[9],columns=["R5"])
        print(R5)
    
    #1.8.modele 7
    try:
        parameters = {'n_estimators':[200,300], 'max_depth':[10,18],'n_jobs':[-1]}
        rfc= RandomForestClassifier()
        m6=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1).fit(X0_train,y0_train)
        proba6=pd.DataFrame(y0_test)
        P = m6.predict_proba(X0_test)[:, 1]
        proba6[probabilite]= P
        R6=mesure(proba6,cible)
        R6=pd.DataFrame(R6)
        R6.rename(columns={cible:"R6"},inplace=True)
        print(R6)
        
    except ValueError:
        R6=pd.DataFrame([0],index=[9],columns=["R6"])
        print(R6)
       
    try:
        coef=[0.14,0.14,0.14,0.14,0.14,0.14,0.14]
        proba7=coef[0]*proba0+coef[1]*proba1+coef[2]*proba2+coef[3]*proba3+coef[4]*proba4+coef[5]*proba5+coef[6]*proba6
        R7=mesure(proba7,cible)
        R7=pd.DataFrame(R7)
        R7.rename(columns={cible:"R7"},inplace=True)
        coef2=coef
        coef=pd.DataFrame(coef2,columns=['valeur'])
        coef.to_csv("coefmodel.csv",sep=';',encoding='utf-8')
        print(R7)
    except ValueError:
        R7=pd.DataFrame([0],index=[9],columns=["R7"])
        print(R7)
    
    try:
        R9=R7
        R9.rename(columns={"R7":"R8"},inplace=True)
        
        for i in xrange(1,100000):
            li=list(np.random.random(7))
            li=li/sum(li)
            coef=list(li)
            proba8=coef[0]*proba0+coef[1]*proba1+coef[2]*proba2+coef[3]*proba3+coef[4]*proba4+coef[5]*proba5+coef[6]*proba6
            R8=mesure(proba8,cible)
            R8=pd.DataFrame(R8)
            R8.rename(columns={cible:"R8"},inplace=True)
            if float(R8["R8"])>float(R9["R8"]):
                R9=R8
                coef2=coef
        coef=pd.DataFrame(coef2,columns=['valeur'])
        coef.to_csv("coefmodel.csv",sep=';',encoding='utf-8')
        print(R9)
    except ValueError:
        R8=pd.DataFrame([0],index=[9],columns=["R8"])
        print(R8)
        
     
    #***********************************III-choix du modele *****************************************
    import cPickle
    plus0="/"
    plus1=cible
    plus2="MOD"
    chemin1=chemin+plus2+plus0+plus1+plus0
    
    if os.path.isdir(chemin1)==True:
        os.chdir(chemin1)
    else:
        os.makedirs(chemin1) 
        os.chdir(chemin1)

        
    R=pd.concat([R0,R1,R2,R3,R4,R5,R6,R7],axis=1)
    RR=R.T
    RR=pd.DataFrame(RR)
    RR.rename(columns=lambda x: 'valeur',inplace=True)
    RR.sort_values(by='valeur',ascending=False,inplace=True)
    RRR=pd.DataFrame(RR.head(1))
    param=RRR.index

     
    if param=="R0":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m0,f)
        export=proba_reco(proba0,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')   
    if param=="R1":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m1,f)
        export=proba_reco(proba1,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    if param=="R2":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m2,f)
        export=proba_reco(proba2,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
        
    if param=="R3":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m3,f)
        export=proba_reco(proba3,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
        
    if param=="R4":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m4,f)
        export=proba_reco(proba4,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R5":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m5,f)
        export=proba_reco(proba5,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R6":
        with open(chemin1+"MODELE.pickle",'wb') as f:
            cPickle.dump(m6,f)
        export=proba_reco(proba6,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
    
    if param=="R7":
        with open(chemin1+"M0.pickle",'wb') as f:
            cPickle.dump(m0,f)
        with open(chemin1+"M1.pickle",'wb') as f:
            cPickle.dump(m1,f)  
        with open(chemin1+"M2.pickle",'wb') as f:
            cPickle.dump(m2,f)
        with open(chemin1+"M3.pickle",'wb') as f:
            cPickle.dump(m3,f)
        with open(chemin1+"M4.pickle",'wb') as f:
            cPickle.dump(m4,f)
        with open(chemin1+"M5.pickle",'wb') as f:
            cPickle.dump(m5,f)
        with open(chemin1+"M6.pickle",'wb') as f:
            cPickle.dump(m6,f)
        export=proba_reco(proba7,cible)
        print(export)
        export.to_csv(chemin1+cible+"_reco_proba.csv",sep=';',encoding='utf-8')
        
    
    print(param)

In [263]:
modelisation(X,y,'cible_un_mois','/home/foueslat/projet-encours/')

         R0
9  1.375182
0.083044982699
OTOTOFOFOFO
0.742468415938
0.285714285714
         R1
9  1.511193
         R2
9  1.375182
0.083044982699
OTOTOFOFOFO
0.742468415938
0.285714285714
        R3
9  1.06192
         R4
9  1.092056
         R5
9  1.536968
         R6
9  1.456075
         R7
9  1.294289
         R8
9  1.577414
9    0.344232
Name: proba_min_decile, dtype: float64
Index([u'R5'], dtype='object')


In [144]:
merged.count()

contactid                 4114
campagne                  4113
campaign                  4113
civilite                  4114
code_postal               4113
country_connexion_name    4113
csp                       4114
date_naissance            4113
debit_cb                  3619
flag_banque_principale    4114
flag_epargne              4114
mailing_accord            4113
namesponsor               2288
nature_cb                 3619
patrimoine                4114
pays                      4113
regime_matrimonial        4114
revenus_annuels           4114
cible_un_mois             4114
dtype: int64

In [ ]:
try:
        parameters = {'n_estimators':[150,300,350], 'n_jobs':[-1]}
        rfc= RandomForestClassifier(n_estimators=200)
        m0=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
        #m0 = m0.fit(X0_train,y0_train)
        rfc_model = rfc.fit(X0_train,y0_train)
        proba0 = pd.DataFrame(y0_test)
        P = m0.predict_proba(X0_test)[:, 1]
        proba0[probabilite] = P
        R0 = mesure(proba0,cible)
        R0 = pd.DataFrame(R0)
        R0.rename(columns={cible:"R0"},inplace=True)
        print(R0)
        
    except ValueError:
        R0=pd.DataFrame([0],index=[9],columns=["R0"])
        print(R0)

In [149]:
from sklearn.cross_validation import train_test_split
from sklearn import linear_model,decomposition
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn import metrics
from sklearn import  grid_search
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    
X0_train,X0_test,y0_train,y0_test=train_test_split(X,y,random_state=42)
rfc= RandomForestClassifier(n_estimators=200)
rfc_model = rfc.fit(X0_train,y0_train)
df_cible_test = pd.DataFrame(y0_test)

In [210]:
r = rfc_model.predict_proba(X0_test)

In [181]:
X0_test.shape, X0_train.shape, y0_test.shape, y0_train.shape

((1029, 18), (3085, 18), (1029,), (3085,))

In [135]:
ar = numpy.array([[1.1, 2, 3.3, 4], [2.7, 10, 5.4, 7], [5.3, 9, 1.5, 15]])
df = pd.DataFrame(ar, index = ['a1', 'a2', 'a3'], columns = ['A', 'B', 'C', 'D'])

In [136]:
df.head()

,A,B,C,D
a1,1.1,2,3.3,4
a2,2.7,10,5.4,7
a3,5.3,9,1.5,15


In [178]:
ar[:, 1]

array([  2.,  10.,   9.])

In [205]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, jaccard_similarity_score, precision_score, f1_score, recall_score

In [214]:
jaccard_similarity_score(y0_test, r[:, 1])

ValueError: Can't handle mix of binary and continuous

In [195]:
accuracy_score(y0_test, r)

0.74829931972789121

In [191]:
confusion_matrix(y0_test, r)

array([[754,  28],
       [231,  16]])

In [194]:
roc_auc_score(y0_test, r)

0.5144858506683786

In [201]:
precision_score(y0_test, r)

0.36363636363636365

In [203]:
f1_score(y0_test, r)

0.10996563573883161

In [206]:
recall_score(y0_test, r)

0.064777327935222673

In [213]:
r[:, 1]

array([ 0.19 ,  0.255,  0.105, ...,  0.07 ,  0.195,  0.185])

In [212]:
r

array([[ 0.81 ,  0.19 ],
       [ 0.745,  0.255],
       [ 0.895,  0.105],
       ..., 
       [ 0.93 ,  0.07 ],
       [ 0.805,  0.195],
       [ 0.815,  0.185]])

In [ ]:
parameters = {'n_estimators':[150,300,350], 'n_jobs':[-1]}
rfc= RandomForestClassifier()
m0=grid_search.GridSearchCV(rfc,parameters,cv=6,n_jobs=-1)
m0 = m0.fit(X0_train,y0_train)
proba0=pd.DataFrame(y0_test)
P = m0.predict_proba(X0_test)[:, 1]
y_pred = m0.predict(X0_test)

proba0[probabilite]= P
R0=mesure(proba0,cible)
R0=pd.DataFrame(R0)
R0.rename(columns={cible:"R0"},inplace=True)
print(R0)

In [ ]:
print('f1 score ' + f1_score(y0_test, r))
print('accuracy ' + accuracy_score(y0_test, r))
print('precision_score  ' + precision_score(y0_test, r))

In [239]:
data_set.count()

campagne                          4113
campaign                          4113
civilite                          4114
code_postal                       4113
country_connexion_name            4113
csp                               4114
date_naissance                    4113
debit_cb                          3619
flag_banque_principale            4114
flag_epargne                      4114
mailing_accord                    4113
namesponsor                       2288
nature_cb                         3619
patrimoine                        4114
pays                              4113
regime_matrimonial                4114
revenus_annuels                   4114
cible_un_mois                     4114
CONTACT_ID                        4114
Pays De Naissance                 4114
Code Postal Ville de Naissance    4114
code_postal_naissance             4114
pays_naissance_iso                4114
dtype: int64

In [268]:
merged.count()

contactid                 4114
campagne                  4113
campaign                  4113
civilite                  4114
code_postal               4113
country_connexion_name    4113
csp                       4114
date_naissance            4113
debit_cb                  3619
flag_banque_principale    4114
flag_epargne              4114
mailing_accord            4113
namesponsor               2288
nature_cb                 3619
patrimoine                4114
pays                      4113
regime_matrimonial        4114
revenus_annuels           4114
cible_un_mois             4114
dtype: int64

In [269]:
X0_train.shape, X0_test.shape, y0_train.shape, y0_test.shape

((3085, 18), (1029, 18), (3085,), (1029,))

In [271]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X0_train, y0_train)
y_pred = lr.predict(X0_test)
print(accuracy_score(y0_test, y_pred))
confusion_matrix(y0_test, y_pred)

0.759961127308


array([[782,   0],
       [247,   0]])

In [270]:
y_pred.shape

(1029,)

In [266]:
best = 1
best_i = 0
for i in xrange(1000):  
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state =i)
    lr = LogisticRegression()
    y_pred = lr.fit(X_train, y_train).predict(X_test)
    _ = accuracy_score(y_test, y_pred)
    if _ < best :
        best = _
        best_i = i
print best_i, best

961 0.731778425656
